<a href="https://colab.research.google.com/github/evan15900/Lee113/blob/main/EX05_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.0 MB/s eta 0:00:00


In [3]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction,
    ImageCarouselTemplate,
    ImageCarouselColumn,
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_Channel_access_token'))
handler = WebhookHandler(userdata.get('LINE_Channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
          template = ConfirmTemplate(
              text='你喜歡中山大學嗎?',
              actions=[
                  MessageAction(label='是',text='我喜歡'),
                  MessageAction(label='否',text='我不喜歡')
              ]
            )
          reply = TemplateMessage(
              alt_text='確認視窗',
              template=template
          )
        elif action == 'carousel':
          carousel_template = CarouselTemplate(
            columns=[
              CarouselColumn(
                thumbnail_image_url='https://cdn.pixabay.com/photo/2020/10/18/13/47/tokyo-tower-5664846_1280.jpg',
                title='東京',
                text='日本的首都東京是政治、文化和經濟的中心地',
                actions=[
                  URIAction(label='旅遊指南', uri='https://www.gltjp.com/zh-hant/article/item/20183/'),
                  MessageAction(label='投票', text='我投東京一票')
                ]
              ),
              CarouselColumn(
                thumbnail_image_url='https://cdn.pixabay.com/photo/2016/11/29/12/55/architecture-1869661_1280.jpg',
                title='京都',
                text='日本著名的文化古都，至今仍保留著許多具有歷史價值的建築物',
                actions=[
                    URIAction(label='旅遊指南', uri='https://www.gltjp.com/zh-hant/article/item/20205/'),
                    MessageAction(label='投票', text='我投京都一票')
                ]
              )
            ]
          )
          reply = TemplateMessage(
            alt_text='輪播視窗',
            template=carousel_template
          )
        elif action == 'image_carousel':
          image_carousel_template = ImageCarouselTemplate(
            columns=[
              ImageCarouselColumn(
                image_url='https://cdn.pixabay.com/photo/2020/10/18/13/47/tokyo-tower-5664846_1280.jpg',
                action=URIAction(
                  label='東京旅遊指南',
                  uri='https://www.gltjp.com/zh-hant/article/item/20183/'
                )
              ),
              ImageCarouselColumn(
                image_url='https://cdn.pixabay.com/photo/2016/11/29/12/55/architecture-1869661_1280.jpg',
                action=URIAction(
                  label='京都旅遊指南',
                  uri='https://www.gltjp.com/zh-hant/article/item/20205/'
                )
              )
            ]
          )

          reply = TemplateMessage(
              alt_text='圖片輪播視窗',
              template=image_carousel_template
          )


        else:
          reply = TextMessage(text='請輸入"confirm/carousel/image_carousel"')

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )


ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3528-34-23-246-28.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 05:34:16] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 05:34:16] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 05:34:24] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 05:34:31] "POST /callback HTTP/1.1" 200 -
